https://www.kaggle.com/datasets/fedesoriano/the-boston-houseprice-data

pointers to explore

can we combine 2 search algos, eg genetic/ randomized and then grid

speed of each algorithm

In [ ]:
  !pip install opendatasets
!pip install pandas
!pip install tpot

In [ ]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time

from tpot import TPOTRegressor
from matplotlib import pyplot as plt

In [ ]:
import opendatasets as od


od.download("https://www.kaggle.com/datasets/fedesoriano/the-boston-houseprice-data")
#32cba7be0f29392dd6e7282c665a3efa#dffdgg#35t5efg


In [ ]:
data=pd.read_csv("/content/the-boston-houseprice-data/boston.csv")
data=data.dropna()
data.head(10)

In [ ]:
# @title ZN


data['ZN'].plot(kind='hist', bins=20, title='ZN')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
# Quick Check data:
data.info()

In [ ]:
data.describe().T

In [ ]:
sns.pairplot(data,kind="reg")


In [ ]:
from xgboost import XGBRegressor

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score#for classification only

In [ ]:
#X=data[["CRIM","INDUS","AGE"]]
X=data.drop(["MEDV"],axis=1)
y=data["MEDV"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)
print("X_train",X_train.shape)
print("X_test",X_test.shape)
print("y_train",y_train.shape)
print("y_test",y_test.shape)

In [ ]:
xgb_model=XGBRegressor().fit(X_train,y_train)

In [ ]:
y_pred=xgb_model.predict(X_test)
mean_squared_error(y_pred,y_test) #lower the better

Model Tuning


-Grid Search

In [ ]:
xgb_params={

    "colsample_bytree":[0.4, 0.6, 0.8, 1.0],
    "learning_rate":[0.01, 0.03, 0.05, 0.07, 0.1, 0.3, 0.5],
    "max_depth":[3, 4, 5, 6, 7, 8],
    'subsample': [0.5, 0.8, 1.0],
    'min_child_weight': [1, 3, 5, 10],
    "n_estimators":[100,200]
}

# colsample_bytree: Number of sample per tree.
# max_depth: Prevent to overfit
#https://www.datasnips.com/blog/2021/7/11/XGBoost-Parameter-Tuning/

In [ ]:
%%time

start_time = time.time()
xgb_gridSearch=GridSearchCV(xgb_model,xgb_params, scoring='neg_mean_absolute_error', cv=10,n_jobs=-1,verbose=0)
xgb_gridSearch.fit(X_train,y_train)
end_time = time.time()

execution_time = end_time - start_time
print(f"Execution time: {execution_time:.6f} seconds")

In [ ]:
xgb_gridSearch.best_params_

In [ ]:
xgb_gridSearch.best_score_

In [ ]:
xgb_best = XGBRegressor(**xgb_gridSearch.best_params_)
xgb_best.fit(X_train,y_train)

In [ ]:
y_pred=xgb_best.predict(X_test)
mean_squared_error(y_pred,y_test)

Random Search

In [ ]:
%%time

from sklearn.model_selection import RandomizedSearchCV
 #Initialize the regressor
xgb=XGBRegressor()


# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=xgb_params,
    n_iter=10,  # Number of random combinations to try
    scoring='neg_mean_absolute_error',  # Choose an appropriate metric
    cv=10,  # Number of cross-validation folds
    verbose=1,
    n_jobs=-1  # Use all available CPU cores
)

# Fit the RandomizedSearchCV object to the training data
random_search.fit(X_train,y_train)

# Print the best hyperparameters and corresponding score
print("Best parameters:", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Refit the model on the entire dataset using the best parameters
#best_xgb_model = xgb(**random_search.best_params_)
#best_xgb_model.fit(X, y)

In [ ]:
best_xgb_model = XGBRegressor(**random_search.best_params_)
best_xgb_model.fit(X_train,y_train)

In [ ]:
y_pred=best_xgb_model.predict(X_test)
print('mean_squared_error:',mean_squared_error(y_test, y_pred))

Genetic Algorithm


In [ ]:
# Define the TPOT configuration for regression
tpot_config = {
    'xgboost.XGBRegressor': {
        'n_estimators': [100, 200],
        'max_depth': [3, 4, 5, 6, 7, 8],
        'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.1, 0.3, 0.5],
       'subsample': [0.5, 0.8, 1.0],
        'colsample_bytree': [0.4, 0.6, 0.8, 1.0, 1.5, 1.7, 2.0],
        'min_child_weight': [1, 3, 5, 10]
    }
}

POPULATION GENERATION COMPARISON

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=10, offspring_size = 6, population_size=10, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=10, offspring_size = 6, population_size=30, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=10, offspring_size = 6, population_size=50, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=10, offspring_size = 6, population_size=70, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=10, offspring_size = 6, population_size=100, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=10, offspring_size = 6, population_size=150, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=10, offspring_size = 6, population_size=200, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=20, offspring_size = 6, population_size=10, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=20, offspring_size = 6, population_size=30, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=20, offspring_size = 6, population_size=50, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=20, offspring_size = 6, population_size=70, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=20, offspring_size = 6, population_size=100, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=20, offspring_size = 6, population_size=150, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=20, offspring_size = 6, population_size=200, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=30, offspring_size = 6, population_size=10, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=30, offspring_size = 6, population_size=30, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=30, offspring_size = 6, population_size=50, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=30, offspring_size = 6, population_size=70, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=30, offspring_size = 6, population_size=100, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=30, offspring_size = 6, population_size=150, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
# Initialize TPOT for regression
tpot = TPOTRegressor(generations=30, offspring_size = 6, population_size=200, cv=10,scoring='neg_mean_absolute_error', config_dict=tpot_config, verbosity=2, n_jobs=-1)
# Fit TPOT to the training data
tpot.fit(X_train, y_train)

# Get the best pipeline (including hyperparameters)
best_pipeline = tpot.fitted_pipeline_

# Get the best score (negative mean absolute error in this case)
best_score = tpot.score(X_test, y_test)

print(f"Best score (NMAE): {best_score:.4f}")

In [ ]:
# Evaluate the best pipeline on the test set using MSE
y_pred = best_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f"Best pipeline: {best_pipeline}")
print(f"Test MSE: {mse:.4f}")

In [ ]:
import matplotlib.pyplot as plt

# Data for the line graphs
populations = [10, 30, 50, 70, 100, 150, 200]
mse_10_gen = [10.1, 8.0, 7.9, 8.0, 9.3, 8.2, 11.4]
mse_20_gen = [9.5, 7.6, 10.0, 8.0, 9.1, 8.8, 9.7]
mse_30_gen = [9.4, 8.6, 8.1, 10.2, 9.2, 8.8, 9.7]

# Create the line graph
plt.figure(figsize=(10, 5))
plt.plot(populations, mse_10_gen, marker='o', label='10 generations')
plt.plot(populations, mse_20_gen, marker='s', label='20 generations')
plt.plot(populations, mse_30_gen, marker='^', label='30 generations')

# Set the x and y axis labels
plt.xlabel('Population')
plt.ylabel('MSE Scores')

# Set the y axis limits
plt.ylim(6, 11)

# Add a legend
plt.legend()

# Add a grid
plt.grid(True)

# Show the plot
plt.show()


GENERATION, POPULATION PAIRS TO TRY

10, 10

10, 30

10, 50

10, 70

10, 100

10, 150

10, 200

do same for generation = 20 and generation = 30